In [1]:
# import jax
# import jax.numpy as jnp

# jax.config.update("jax_compilation_cache_dir", "../jax-caches")
# jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
# jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../"))

# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.25"
# os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
# from desc import set_device
# set_device("gpu")

In [3]:
# from desc import set_device, _set_cpu_count

# num_device = 2
# _set_cpu_count(num_device)
# set_device("cpu", num_device=num_device)

In [4]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [5]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic
from desc.compute.data_index import register_compute_fun
from desc.optimize.utils import solve_triangular_regularized

print_backend_info()

DESC version=0.14.2+105.gbb4c46f42.
Using JAX backend: jax version=0.6.0, jaxlib version=0.6.0, dtype=float64.
Using device: CPU, with 11.93 GB available memory.


In [ ]:
from desc.compute.geom_utils import rotation_matrix


def safearccos(x):
    """Like jnp.arccos, but without nan gradient at x=1."""
    safe_x = jnp.where(jnp.abs(x) == 1, 0, x)
    return jnp.where(jnp.abs(x) == 1, sign(x) * jnp.inf, jnp.arccos(safe_x))


def test(normal):
    Zaxis = jnp.array([0.0, 0.0, 1.0])  # 2D curve in X-Y plane has normal = +Z axis
    axis = cross(Zaxis, normal)
    dotprod = dot(Zaxis, safenormalize(normal))
    angle = safearccos(dotprod)
    A = jnp.where(
        jnp.allclose(dotprod, -1.0),
        jnp.diag(jnp.array([1.0, -1.0, -1.0])),
        rotation_matrix(axis, angle),
    )  # Handle the case of 180 degree rotation
    return jnp.sum(A@normal)

normal = jnp.array([0.0, 0., -2.0])  # Normal vector of the curve in X-Y plane
print(test(normal), jax.grad(test)(normal))

2.0 [ 1. -1. -1.]


In [59]:
Zaxis = jnp.array([0.0, 0.0, 1.0])  # 2D curve in X-Y plane has normal = +Z axis
axis = cross(Zaxis, normal)
dotprod = dot(Zaxis, safenormalize(normal))
angle = safearccos(dotprod)
A = rotation_matrix(axis, angle)
normal = jnp.array([0.0, 0.0, -2.0])  # Normal vector of the curve in X-Y plane
A @ normal, A

(Array([ 0.,  0., -2.], dtype=float64),
 Array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], dtype=float64))